In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import pmdarima as pm
import matplotlib.pyplot as plt

In [2]:
Location = f'D:\pothole\Pothole_Map.csv'
df = pd.read_csv(Location)
df.dropna(subset=['Created Date', 'Latitude', 'Longitude'], inplace=True)
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'])
sortdf = df[df['Resolution Description'] ==  'The Department of Transportation inspected this complaint and repaired the problem.'].copy()
#getting rid of outlier
sortdf['Created Day'] = sortdf['Created Date'].dt.date
sortdf['xOutlier'] = (abs(sortdf['Latitude'] - sortdf['Latitude'].mean()) > 1.96*sortdf['Latitude'].std())
sortdf['yOutlier'] = (abs(sortdf['Longitude'] - sortdf['Longitude'].mean()) > 1.96*sortdf['Longitude'].std())
sortdf = sortdf[sortdf['xOutlier']==False]
sortdf = sortdf[sortdf['yOutlier']==False]
#sortdf.to_csv(r'C:\pothole\sortedpotholefile.csv')
sortdf['month'] = sortdf['Created Date'].dt.to_period('M')
sortdf['month'] = sortdf['month'].dt.strftime('%Y-%m')
sortdf['Date'] = pd.to_datetime(sortdf['Created Day']).dt.to_period('m')
sortdf['value'] =1
date = sortdf[['Date', 'value']].groupby(['Date'],as_index=False).sum()
date['year'] = date['Date'].dt.year
date['month'] = date['Date'].dt.month
#print(date)
x = sortdf['Latitude'].to_numpy()
y = sortdf['Longitude'].to_numpy()
deltaX = (max(x) - min(x))/10
deltaY = (max(y) - min(y))/10
xmin = min(x) - deltaX
xmax = max(x) + deltaX
ymin = min(y) - deltaY
ymax = max(y) + deltaY
ydiff2 = ymax - ymin
xdiff2 = xmax - xmin
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])

1.5466091127799283e-07


In [3]:
def densev(input):
    m1 = sortdf[sortdf['month'] == input]
    sx = m1['Latitude'].to_numpy()
    sy = m1['Longitude'].to_numpy()    
    values = np.vstack([sx, sy])
    kernel = st.gaussian_kde(values)
    sf = np.reshape(kernel(positions).T, xx.shape)
    return sx, sy, sf

In [4]:
sortdf

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Segment,Latitude,Longitude,Location,Created Day,xOutlier,yOutlier,month,Date,value
0,32325294,2016-01-02 08:49:18,2016-01-03 04:25:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10025.0,BROADWAY,...,NaN,40.799442,-73.968371,"(40.7994424593551, -73.96837112066619)",2016-01-02,False,False,2016-01,2016-01,1
1,32718369,2016-02-18 01:05:38,2016-02-18 22:57:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10025.0,CENTRAL PARK WEST,...,NaN,40.794541,-73.962584,"(40.79454138582705, -73.96258409624588)",2016-02-18,False,False,2016-02,2016-02,1
3,33121198,2016-04-12 13:51:26,2016-04-13 09:20:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10025.0,BROADWAY,...,NaN,40.799442,-73.968371,"(40.7994424593551, -73.96837112066619)",2016-04-12,False,False,2016-04,2016-04,1
4,33437277,2016-05-24 14:43:02,2016-05-25 10:44:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10023.0,CENTRAL PARK WEST,...,NaN,40.771468,-73.979425,"(40.771467896435546, -73.97942477301613)",2016-05-24,False,False,2016-05,2016-05,1
5,33561528,2016-06-09 13:43:17,2016-06-10 10:00:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10024.0,CENTRAL PARK WEST,...,NaN,40.787541,-73.967691,"(40.78754112159358, -73.96769064758249)",2016-06-09,False,False,2016-06,2016-06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,45855521,2020-03-17 00:17:20,2020-03-18 23:07:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10024.0,387 AMSTERDAM AVENUE,...,NaN,40.782804,-73.978565,"(40.7828035026212, -73.97856547811351)",2020-03-17,False,False,2020-03,2020-03,1
3110,45896015,2020-03-25 13:51:35,2020-03-26 22:40:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10025.0,NaN,...,NaN,40.795252,-73.973209,"(40.795252485299606, -73.97320906036077)",2020-03-25,False,False,2020-03,2020-03,1
3111,45949695,2020-04-05 10:18:43,2020-04-07 11:50:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10023.0,1901 BROADWAY,...,NaN,40.771515,-73.982172,"(40.77151501485562, -73.98217221573978)",2020-04-05,False,False,2020-04,2020-04,1
3112,45973750,2020-04-08 00:53:44,2020-04-09 23:30:00,DOT,Department of Transportation,Street Condition,Pothole,NaN,10024.0,2163 BROADWAY,...,NaN,40.781917,-73.981018,"(40.781917383347114, -73.98101755698409)",2020-04-08,False,False,2020-04,2020-04,1


In [13]:
rx1, ry1, rf1 = densev('2020-03')
rasterdata = pd.DataFrame(columns = ['density','latitude','longitude'])
for i in range(0,100):
    for j in range(0,100):
        #df2 = {'density':rf1[i,j],'longitude':xx[i,1],'latitude':yy[j,1]}
        #print(df2)
        rasterdata.loc[len(rasterdata.index)]= [rf1[i,j],xx[i,1], yy[1,j]]

In [14]:
rasterdata.to_csv(r'D:\pothole\rasterdata2020-03.csv')